In [14]:
import pandas as pd

In [15]:
# read data_base
df = pd.read_csv('mental_health_monitoring_dataset.csv')
display(df)

,Timestamp,Heart_Rate,Blood_Pressure_Systolic,Blood_Pressure_Diastolic,Skin_Temperature,Galvanic_Skin_Response,Respiration_Rate,Sleep_Duration,Activity_Levels,Mood,...,Fuel_Consumption,Average_Speed,Work_Hours,Job_Stressors,Location_Latitude,Location_Longitude,Stress_Level,Mental_Health_History,Resilience_Factors,Mental_Health_Status
0,2019-01-01 00:00:00,98,177,99,36.854833,0.603660,20,4,4945,Happy,...,17.568925,78,12,5,51.002436,44.895265,Low,Yes,5,Normal
1,2019-01-01 01:00:00,111,133,86,37.239401,0.934820,24,5,6221,Happy,...,10.295241,25,8,9,42.839887,75.276677,Low,No,2,Moderate Stress
2,2019-01-01 02:00:00,88,154,113,36.209588,0.035483,28,1,6426,Anxious,...,8.515171,37,15,9,68.836583,20.795174,Low,No,4,Depression
3,2019-01-01 03:00:00,74,162,60,36.087569,0.653249,25,4,14435,Happy,...,13.774633,39,15,5,51.752928,89.835756,Low,No,1,Normal
4,2019-01-01 04:00:00,102,160,81,36.593949,0.756138,15,6,4334,Happy,...,18.747632,87,11,1,98.613250,86.654677,Low,No,2,Moderate Stress
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52580,2024-12-30 20:00:00,86,120,92,36.371463,0.817159,18,2,16712,Sad,...,8.554723,44,12,2,45.538176,91.829336,Low,Yes,7,Moderate Stress
52581,2024-12-30 21:00:00,81,168,118,34.847808,0.087909,21,0,8975,Anxious,...,16.481642,80,9,5,33.411870,52.294238,Medium,No,5,Moderate Stress
52582,2024-12-30 22:00:00,61,149,119,36.782976,0.539182,20,6,8770,Neutral,...,5.111862,23,14,1,67.402028,21.529958,Low,No,1,Normal
52583,2024-12-30 23:00:00,66,133,72,37.084081,0.103317,16,4,14148,Happy,...,7.017609,83,8,1,16.589861,50.978252,Low,No,7,Normal


In [16]:
# remove rows with nan values
original_len = len(df)
df = df.dropna()
print(f"number of removed rows: {original_len - len(df)}")
display(df)

number of removed rows: 0


,Timestamp,Heart_Rate,Blood_Pressure_Systolic,Blood_Pressure_Diastolic,Skin_Temperature,Galvanic_Skin_Response,Respiration_Rate,Sleep_Duration,Activity_Levels,Mood,...,Fuel_Consumption,Average_Speed,Work_Hours,Job_Stressors,Location_Latitude,Location_Longitude,Stress_Level,Mental_Health_History,Resilience_Factors,Mental_Health_Status
0,2019-01-01 00:00:00,98,177,99,36.854833,0.603660,20,4,4945,Happy,...,17.568925,78,12,5,51.002436,44.895265,Low,Yes,5,Normal
1,2019-01-01 01:00:00,111,133,86,37.239401,0.934820,24,5,6221,Happy,...,10.295241,25,8,9,42.839887,75.276677,Low,No,2,Moderate Stress
2,2019-01-01 02:00:00,88,154,113,36.209588,0.035483,28,1,6426,Anxious,...,8.515171,37,15,9,68.836583,20.795174,Low,No,4,Depression
3,2019-01-01 03:00:00,74,162,60,36.087569,0.653249,25,4,14435,Happy,...,13.774633,39,15,5,51.752928,89.835756,Low,No,1,Normal
4,2019-01-01 04:00:00,102,160,81,36.593949,0.756138,15,6,4334,Happy,...,18.747632,87,11,1,98.613250,86.654677,Low,No,2,Moderate Stress
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52580,2024-12-30 20:00:00,86,120,92,36.371463,0.817159,18,2,16712,Sad,...,8.554723,44,12,2,45.538176,91.829336,Low,Yes,7,Moderate Stress
52581,2024-12-30 21:00:00,81,168,118,34.847808,0.087909,21,0,8975,Anxious,...,16.481642,80,9,5,33.411870,52.294238,Medium,No,5,Moderate Stress
52582,2024-12-30 22:00:00,61,149,119,36.782976,0.539182,20,6,8770,Neutral,...,5.111862,23,14,1,67.402028,21.529958,Low,No,1,Normal
52583,2024-12-30 23:00:00,66,133,72,37.084081,0.103317,16,4,14148,Happy,...,7.017609,83,8,1,16.589861,50.978252,Low,No,7,Normal


In [17]:
# display duplicates
df[df.duplicated()].shape[0]

0

In [18]:
# display the remain values in the column Sleep_Duration
df['Sleep_Duration'].unique()

array([4, 5, 1, 6, 3, 9, 7, 8, 0, 2], dtype=int64)

In [19]:
# delete rows with outliers values, Sleep_Duration below 1
original_len = len(df)
df = df[df['Sleep_Duration'] >= 1]
print(f"number of removed rows: {original_len - len(df)}")
display(df)

number of removed rows: 5240


,Timestamp,Heart_Rate,Blood_Pressure_Systolic,Blood_Pressure_Diastolic,Skin_Temperature,Galvanic_Skin_Response,Respiration_Rate,Sleep_Duration,Activity_Levels,Mood,...,Fuel_Consumption,Average_Speed,Work_Hours,Job_Stressors,Location_Latitude,Location_Longitude,Stress_Level,Mental_Health_History,Resilience_Factors,Mental_Health_Status
0,2019-01-01 00:00:00,98,177,99,36.854833,0.603660,20,4,4945,Happy,...,17.568925,78,12,5,51.002436,44.895265,Low,Yes,5,Normal
1,2019-01-01 01:00:00,111,133,86,37.239401,0.934820,24,5,6221,Happy,...,10.295241,25,8,9,42.839887,75.276677,Low,No,2,Moderate Stress
2,2019-01-01 02:00:00,88,154,113,36.209588,0.035483,28,1,6426,Anxious,...,8.515171,37,15,9,68.836583,20.795174,Low,No,4,Depression
3,2019-01-01 03:00:00,74,162,60,36.087569,0.653249,25,4,14435,Happy,...,13.774633,39,15,5,51.752928,89.835756,Low,No,1,Normal
4,2019-01-01 04:00:00,102,160,81,36.593949,0.756138,15,6,4334,Happy,...,18.747632,87,11,1,98.613250,86.654677,Low,No,2,Moderate Stress
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52578,2024-12-30 18:00:00,114,137,84,37.106297,0.377905,14,6,6767,Sad,...,16.159221,35,9,7,76.503918,24.779305,Medium,No,8,Moderate Stress
52580,2024-12-30 20:00:00,86,120,92,36.371463,0.817159,18,2,16712,Sad,...,8.554723,44,12,2,45.538176,91.829336,Low,Yes,7,Moderate Stress
52582,2024-12-30 22:00:00,61,149,119,36.782976,0.539182,20,6,8770,Neutral,...,5.111862,23,14,1,67.402028,21.529958,Low,No,1,Normal
52583,2024-12-30 23:00:00,66,133,72,37.084081,0.103317,16,4,14148,Happy,...,7.017609,83,8,1,16.589861,50.978252,Low,No,7,Normal


In [20]:
# delete unrelevante columns
df = df.drop(['Location_Longitude', 'Location_Latitude', 'Fuel_Consumption', 'Timestamp', 'Galvanic_Skin_Response'], axis=1)
display(df)

,Heart_Rate,Blood_Pressure_Systolic,Blood_Pressure_Diastolic,Skin_Temperature,Respiration_Rate,Sleep_Duration,Activity_Levels,Mood,Cognitive_Load,Social_Interaction,Driving_Conditions,Route_Duration,Average_Speed,Work_Hours,Job_Stressors,Stress_Level,Mental_Health_History,Resilience_Factors,Mental_Health_Status
0,98,177,99,36.854833,20,4,4945,Happy,5,7,Moderate,6,78,12,5,Low,Yes,5,Normal
1,111,133,86,37.239401,24,5,6221,Happy,4,1,Moderate,5,25,8,9,Low,No,2,Moderate Stress
2,88,154,113,36.209588,28,1,6426,Anxious,3,8,Good,20,37,15,9,Low,No,4,Depression
3,74,162,60,36.087569,25,4,14435,Happy,4,4,Poor,12,39,15,5,Low,No,1,Normal
4,102,160,81,36.593949,15,6,4334,Happy,3,6,Moderate,22,87,11,1,Low,No,2,Moderate Stress
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52578,114,137,84,37.106297,14,6,6767,Sad,3,8,Moderate,9,35,9,7,Medium,No,8,Moderate Stress
52580,86,120,92,36.371463,18,2,16712,Sad,2,6,Good,13,44,12,2,Low,Yes,7,Moderate Stress
52582,61,149,119,36.782976,20,6,8770,Neutral,3,2,Poor,15,23,14,1,Low,No,1,Normal
52583,66,133,72,37.084081,16,4,14148,Happy,5,1,Moderate,16,83,8,1,Low,No,7,Normal


In [21]:
# delete rows with outliers values
# temperature is below 35 in Skin_Temperature
original_len = len(df)
df = df[df['Skin_Temperature'] >= 35]
print(f"number of removed rows: {original_len - len(df)}")
display(df)

number of removed rows: 53


,Heart_Rate,Blood_Pressure_Systolic,Blood_Pressure_Diastolic,Skin_Temperature,Respiration_Rate,Sleep_Duration,Activity_Levels,Mood,Cognitive_Load,Social_Interaction,Driving_Conditions,Route_Duration,Average_Speed,Work_Hours,Job_Stressors,Stress_Level,Mental_Health_History,Resilience_Factors,Mental_Health_Status
0,98,177,99,36.854833,20,4,4945,Happy,5,7,Moderate,6,78,12,5,Low,Yes,5,Normal
1,111,133,86,37.239401,24,5,6221,Happy,4,1,Moderate,5,25,8,9,Low,No,2,Moderate Stress
2,88,154,113,36.209588,28,1,6426,Anxious,3,8,Good,20,37,15,9,Low,No,4,Depression
3,74,162,60,36.087569,25,4,14435,Happy,4,4,Poor,12,39,15,5,Low,No,1,Normal
4,102,160,81,36.593949,15,6,4334,Happy,3,6,Moderate,22,87,11,1,Low,No,2,Moderate Stress
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52578,114,137,84,37.106297,14,6,6767,Sad,3,8,Moderate,9,35,9,7,Medium,No,8,Moderate Stress
52580,86,120,92,36.371463,18,2,16712,Sad,2,6,Good,13,44,12,2,Low,Yes,7,Moderate Stress
52582,61,149,119,36.782976,20,6,8770,Neutral,3,2,Poor,15,23,14,1,Low,No,1,Normal
52583,66,133,72,37.084081,16,4,14148,Happy,5,1,Moderate,16,83,8,1,Low,No,7,Normal


In [22]:
# remove records where the values in the columns Mood and Stress_Level are logically incompatible
# 1. Mood is 'High' and Stress_Level is either 'Anxious', 'Sad', or 'Irritable'
# 2. Mood is 'Low' and Stress_Level is 'Happy'
original_len = len(df)
df = df[
    ~((df['Mood'] == 'High') & df['Stress_Level'].isin(['Anxious', 'Sad', 'Irritable'])) &
    ~((df['Mood'] == 'Low') & (df['Stress_Level'] == 'Happy'))
]
print(f"number of removed rows: {original_len - len(df)}")
display(df)

number of removed rows: 0


,Heart_Rate,Blood_Pressure_Systolic,Blood_Pressure_Diastolic,Skin_Temperature,Respiration_Rate,Sleep_Duration,Activity_Levels,Mood,Cognitive_Load,Social_Interaction,Driving_Conditions,Route_Duration,Average_Speed,Work_Hours,Job_Stressors,Stress_Level,Mental_Health_History,Resilience_Factors,Mental_Health_Status
0,98,177,99,36.854833,20,4,4945,Happy,5,7,Moderate,6,78,12,5,Low,Yes,5,Normal
1,111,133,86,37.239401,24,5,6221,Happy,4,1,Moderate,5,25,8,9,Low,No,2,Moderate Stress
2,88,154,113,36.209588,28,1,6426,Anxious,3,8,Good,20,37,15,9,Low,No,4,Depression
3,74,162,60,36.087569,25,4,14435,Happy,4,4,Poor,12,39,15,5,Low,No,1,Normal
4,102,160,81,36.593949,15,6,4334,Happy,3,6,Moderate,22,87,11,1,Low,No,2,Moderate Stress
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52578,114,137,84,37.106297,14,6,6767,Sad,3,8,Moderate,9,35,9,7,Medium,No,8,Moderate Stress
52580,86,120,92,36.371463,18,2,16712,Sad,2,6,Good,13,44,12,2,Low,Yes,7,Moderate Stress
52582,61,149,119,36.782976,20,6,8770,Neutral,3,2,Poor,15,23,14,1,Low,No,1,Normal
52583,66,133,72,37.084081,16,4,14148,Happy,5,1,Moderate,16,83,8,1,Low,No,7,Normal


In [23]:
# change text values to numeric values in the columns Driving_Conditions, Stress_Level, Mental_Health_History, Mental_Health_Status, Mood
from sklearn.preprocessing import LabelEncoder
columns_to_encode = ['Driving_Conditions', 'Stress_Level', 'Mental_Health_History', 'Mental_Health_Status', 'Mood']
# for transform new data later with same encodings or reverse the numeric values back to text
encoders = {}
mapping_text_to_numeric = {}

for col in columns_to_encode:
    encoders[col] = LabelEncoder()
    df[col] = encoders[col].fit_transform(df[col])
    # mapping dictionary
    mapping_text_to_numeric = dict(zip(encoders[col].classes_, encoders[col].transform(encoders[col].classes_)))

print(df)

       Heart_Rate  Blood_Pressure_Systolic  Blood_Pressure_Diastolic  \
0              98                      177                        99   
1             111                      133                        86   
2              88                      154                       113   
3              74                      162                        60   
4             102                      160                        81   
...           ...                      ...                       ...   
52578         114                      137                        84   
52580          86                      120                        92   
52582          61                      149                       119   
52583          66                      133                        72   
52584          75                      112                       115   

       Skin_Temperature  Respiration_Rate  Sleep_Duration  Activity_Levels  \
0             36.854833                20               4

In [24]:
# categorize numeric values in the column Activity_Levels
import numpy as np

df['Activity_Levels'] = np.where(df['Activity_Levels'] <= 3000, 1, np.where(df['Activity_Levels'] <= 10000, 2, 3))
display(df)

,Heart_Rate,Blood_Pressure_Systolic,Blood_Pressure_Diastolic,Skin_Temperature,Respiration_Rate,Sleep_Duration,Activity_Levels,Mood,Cognitive_Load,Social_Interaction,Driving_Conditions,Route_Duration,Average_Speed,Work_Hours,Job_Stressors,Stress_Level,Mental_Health_History,Resilience_Factors,Mental_Health_Status
0,98,177,99,36.854833,20,4,2,1,5,7,1,6,78,12,5,1,1,5,4
1,111,133,86,37.239401,24,5,2,1,4,1,1,5,25,8,9,1,0,2,3
2,88,154,113,36.209588,28,1,2,0,3,8,0,20,37,15,9,1,0,4,1
3,74,162,60,36.087569,25,4,3,1,4,4,2,12,39,15,5,1,0,1,4
4,102,160,81,36.593949,15,6,2,1,3,6,1,22,87,11,1,1,0,2,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52578,114,137,84,37.106297,14,6,2,4,3,8,1,9,35,9,7,2,0,8,3
52580,86,120,92,36.371463,18,2,3,4,2,6,0,13,44,12,2,1,1,7,3
52582,61,149,119,36.782976,20,6,2,3,3,2,2,15,23,14,1,1,0,1,4
52583,66,133,72,37.084081,16,4,3,1,5,1,1,16,83,8,1,1,0,7,4


In [25]:
# normalize the numeric values in the columns Heart_Rate, Blood_Pressure_systolic, Blood_Pressure_diastolic
from sklearn.preprocessing import MinMaxScaler

columns_to_normalize = ['Heart_Rate', 'Blood_Pressure_Systolic', 'Blood_Pressure_Diastolic']
scaler = MinMaxScaler()
df[columns_to_normalize] = scaler.fit_transform(df[columns_to_normalize])

display(df)

,Heart_Rate,Blood_Pressure_Systolic,Blood_Pressure_Diastolic,Skin_Temperature,Respiration_Rate,Sleep_Duration,Activity_Levels,Mood,Cognitive_Load,Social_Interaction,Driving_Conditions,Route_Duration,Average_Speed,Work_Hours,Job_Stressors,Stress_Level,Mental_Health_History,Resilience_Factors,Mental_Health_Status
0,0.644068,0.977528,0.661017,36.854833,20,4,2,1,5,7,1,6,78,12,5,1,1,5,4
1,0.864407,0.483146,0.440678,37.239401,24,5,2,1,4,1,1,5,25,8,9,1,0,2,3
2,0.474576,0.719101,0.898305,36.209588,28,1,2,0,3,8,0,20,37,15,9,1,0,4,1
3,0.237288,0.808989,0.000000,36.087569,25,4,3,1,4,4,2,12,39,15,5,1,0,1,4
4,0.711864,0.786517,0.355932,36.593949,15,6,2,1,3,6,1,22,87,11,1,1,0,2,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52578,0.915254,0.528090,0.406780,37.106297,14,6,2,4,3,8,1,9,35,9,7,2,0,8,3
52580,0.440678,0.337079,0.542373,36.371463,18,2,3,4,2,6,0,13,44,12,2,1,1,7,3
52582,0.016949,0.662921,1.000000,36.782976,20,6,2,3,3,2,2,15,23,14,1,1,0,1,4
52583,0.101695,0.483146,0.203390,37.084081,16,4,3,1,5,1,1,16,83,8,1,1,0,7,4
